In [4]:
import config
from soynlp.hangle import decompose, compose

for c in 'aㄱ가ㅏ':
    print('{} -> {}'.format(c, decompose(c)))

a -> None
ㄱ -> ('ㄱ', ' ', ' ')
가 -> ('ㄱ', 'ㅏ', ' ')
ㅏ -> (' ', 'ㅏ', ' ')


In [12]:
import config
from soynlp.hangle import decompose, compose
from soynlp.normalizer import remove_doublespace

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

In [13]:
s = '가나다ㅁ  asf 2 아하핫 1!!'
encode(s)

'ㄱㅏ-ㄴㅏ-ㄷㅏ- ㅇㅏ-ㅎㅏ-ㅎㅏㅅ'

In [14]:
decode(encode(s))

'가나다 아하핫'

In [21]:
from navermovie_comments import get_movie_comments_path

for large in [False, True]:
    inpath = get_movie_comments_path(large=large)
    outpath = inpath[:-4] + '_fasttext.txt'
    with open(inpath, encoding='utf-8') as fi:
        with open(outpath, 'w', encoding='utf-8') as fo:
            for line in fi:
                idx, text, rate = line.strip().split('\t')
                text = encode(text)
                fo.write('{}\t{}\t{}\n'.format(idx, text, rate))